In [40]:
import json
import cv2
# 读取train_IDs_tv.txt 文件
# 转换为coco格式
# 保存为train.json

def convert(file_name):
    image_id = file_name.split('_')[0] + file_name.split('_')[1].split('.')[0]
    image_id = int(image_id)
    return image_id

def convert2Coco(): 
    # 读取train_IDs_tv.txt 文件
    with open('TV-Hand_annotations.txt', 'r') as f:
        lines = f.readlines()
        lines = [line.strip() for line in lines]
    # 转换为coco格式
    coco = {}
    coco['images'] = []
    coco['categories'] = []
    coco['annotations'] = []

    # 类别
    category = {}
    category['id'] = 1
    category['name'] = 'hand'
    coco['categories'].append(category)
    # 图片
    # 相同的图片只添加一次
    old_image_name = ''
    for line in lines:
        image = {}
        image['file_name'] = line.split(',')[0]
        if image['file_name'] == old_image_name:
            continue
        old_image_name = image['file_name']
        # cv2读取图片，获取图片的高和宽
        img = cv2.imread('images/'+image['file_name'])
        image['height'] = img.shape[0]
        image['width'] = img.shape[1]
        # image['height'] = 720
        # image['width'] = 1280
        # TV-Hand_Images/000011_004.jpg --> 000011004 --> 11004
        
        image['id'] = convert(image['file_name'])
        coco['images'].append(image)

    # 标注

    i = 1
    for line in lines:
        annotation = {}
        # TV-Hand_Images/000011_004.jpg --> 000011004 --> 11004
        annotation['image_id'] = convert(line.split(',')[0])
        annotation['category_id'] = 1
        # annotation['segmentation'] = []
        xmin = min(line.split(',')[5], line.split(',')[7], line.split(',')[9], line.split(',')[11])
        xmax = max(line.split(',')[5], line.split(',')[7], line.split(',')[9], line.split(',')[11])
        ymin = min(line.split(',')[6], line.split(',')[8], line.split(',')[10], line.split(',')[12])
        ymax = max(line.split(',')[6], line.split(',')[8], line.split(',')[10], line.split(',')[12])
        annotation['bbox'] = [int(xmin), int(ymin), int(xmax)-int(xmin), int(ymax)-int(ymin)]
        annotation['area'] = (int(xmax)-int(xmin)) * (int(ymax)-int(ymin))
        annotation['iscrowd'] = 0
        annotation['id'] = i
        coco['annotations'].append(annotation)
        i += 1

    # 删除area<=0的标注和图片
    for i in range(len(coco['annotations'])-1, -1, -1):
        if coco['annotations'][i]['area'] <= 0:
            image_id = coco['annotations'][i]['image_id']
            del coco['annotations'][i]
            for j in range(len(coco['images'])-1, -1, -1):
                if coco['images'][j]['id'] == image_id:
                    del coco['images'][j]
                    break
    
    # 保存为train.json
    with open('label.json', 'w') as f:
        json.dump(coco, f)
convert2Coco()

In [38]:
import shutil
# 将label.json分为train.json和val.json
# train.json: 80%
# val.json: 20%
def split_train_val():
    with open('label.json', 'r') as f:
        label = json.load(f)
    train = {}
    val = {}
    train['images'] = []
    train['categories'] = []
    train['annotations'] = []
    val['images'] = []
    val['categories'] = []
    val['annotations'] = []
    train['categories'] = label['categories']
    val['categories'] = label['categories']
    # 80%的图片
    train['images'] = label['images'][:int(len(label['images'])*0.7)]
    # 20%的图片
    val['images'] = label['images'][int(len(label['images'])*0.7):]
    
    # 根据train['images'] 和 val['images']的id，找到对应的标注
    for annotation in label['annotations']:
        if annotation['image_id'] in [image['id'] for image in train['images']]:
            train['annotations'].append(annotation)
        elif annotation['image_id'] in [image['id'] for image in val['images']]:
            val['annotations'].append(annotation)

    with open('train.json', 'w') as f:
        json.dump(train, f)
    with open('val.json', 'w') as f:
        json.dump(val, f)
    # 将train.json和val.json移动到annotations文件夹下
    shutil.move('train.json', 'annotations/train.json')
    shutil.move('val.json', 'annotations/val.json')
    # shutil.move('label.json', 'annotations/label.json')
split_train_val()